In [ ]:
import matplotlib.pyplot as plt  # type: ignore
from dotenv import load_dotenv
import sqlalchemy as sq  # type: ignore
import seaborn as sns  # type: ignore
import pandas as pd  # type: ignore
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [ ]:
soilQuery = sq.text(
    """
    SELECT * FROM public.agg_soil_data;
    """
)

soilDf = pd.read_sql_query(soilQuery, conn)

In [ ]:
ergotQuery = sq.text("SELECT * FROM public.agg_ergot_samples")

ergotDF = pd.read_sql_query(ergotQuery, conn)

In [ ]:
final_df = soilDf.merge(ergotDF, on=["district"])

In [ ]:
final_df.columns

In [ ]:
final_df.drop(columns=["district", "year"], inplace=True)

In [ ]:
corr = final_df.corr()

In [ ]:
corr

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    vmin=-1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

In [ ]:
sns.pairplot(final_df[["has_ergot", "sum_severity", "avg_calcium_ph", "avg_proj_ph"]])
plt.show()

In [ ]:
sns.pairplot(
    final_df[["has_ergot", "sum_severity", "avg_total_sand", "avg_water_reten_10"]]
)
plt.show()

In [ ]:
db.cleanup()